## COVID has had a huge impact on vehicle miles traveled (VMT) so being in the auto industry, I wanted to analyze some trends with the data. My data source appears to be a third-party site but they do say the source of the data is the Federal Highway Administration (FHA) so I feel okay about the data. This webscrape / analysis is strictly related to data in the United States.

In [ ]:
import requests
from lxml import html
import pandas as pd
import bs4
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime
from pandas import Timestamp
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, interactive, fixed
from IPython.display import display
import sweetviz
%matplotlib inline

# Webscrape

In [ ]:
VMT_URL = 'https://ycharts.com/indicators/us_monthly_total_vehicle_miles_traveled'
VMT_page = requests.get(VMT_URL).text

In [ ]:
print(VMT_page)

In [ ]:
# Make some soup and make it taste good...well, make it pretty in this case

soup_VMT = bs4.BeautifulSoup(VMT_page, 'lxml')
 
print(soup_VMT.prettify())

In [ ]:
# Pulling the title of the webpage just to make sure things are working

VMT_title = soup_VMT.title
print(VMT_title.text)

In [ ]:
# Find the number of tables on the page

VMT_tables = soup_VMT.find_all('table')
len(VMT_tables)

#### There are 12 tables on the page so I am going to capture them all and analyze their contents.

In [ ]:
# Capture the tables

first_VMT_table = VMT_tables[0]
second_VMT_table = VMT_tables[1]
third_VMT_table = VMT_tables[2]
fourth_VMT_table = VMT_tables[3]
fifth_VMT_table = VMT_tables[4]
sixth_VMT_table = VMT_tables[5]
seventh_VMT_table = VMT_tables[6]
eighth_VMT_table = VMT_tables[7]
ninth_VMT_table = VMT_tables[8]
tenth_VMT_table = VMT_tables[9]
eleventh_VMT_table = VMT_tables[10]
twelfth_VMT_table = VMT_tables[11]

In [ ]:
# Find the first table and display contant 

first_VMT_table.contents

#### Nope, not what I want.

In [ ]:
# Find the second table and display contant 

second_VMT_table.contents

#### Still not what I want.

In [ ]:
# Went back to the HTML output and found the table I was looking for; it is the sixth table on the page

sixth_VMT_table.contents

#### Hmmm...I know my date range goes back to 2017 so why does it stop at March 2019? I know, my other data must be in another table. Let's try the seventh table.

In [ ]:
# Hope the second half of my data is in another table

seventh_VMT_table.contents

#### Ta-da, there is the other half. Looking at the HTML output, my rows are nested within "tr" and my headers "Date" and "Values" are nested in 'th' so let's pull all that out.

In [ ]:
# Pull the rows of the table

VMT_table6_rows = sixth_VMT_table.find_all('tr')
VMT_table7_rows = seventh_VMT_table.find_all('tr')

In [ ]:
print(VMT_table6_rows[:24])

In [ ]:
print(VMT_table7_rows[:24])

In [ ]:
# Get the sixth table header row

t6_header_row = VMT_table6_rows[0]
t6_header_row

In [ ]:
# Get the seventh table header row

t7_header_row = VMT_table7_rows[0]
t7_header_row

In [ ]:
# Access header text in sixth table

ths = t6_header_row.find_all('th')
header6 = [th.text.strip() for th in ths]
print(header6)
print(len(header6))

In [ ]:
# Access header text in seventh table

ths = t7_header_row.find_all('th')
header7 = [th.text.strip() for th in ths]
print(header7)
print(len(header7))

In [ ]:
# Put the data in the sixth table in to a data frame

VMT_t6_data_rows = []
for tr in VMT_table6_rows:
    tds = tr.find_all('td')
    row = [td.text.replace("\n", "").strip() for td in tds]
    if row:
        VMT_t6_data_rows.append(row)
        
VMT_t6 = pd.DataFrame(VMT_t6_data_rows, columns = header6)
VMT_t6

In [ ]:
# Put the data in the seventh table in to a data frame

VMT_t7_data_rows = []
for tr in VMT_table7_rows:
    tds = tr.find_all('td')
    row = [td.text.replace("\n", "").strip() for td in tds]
    if row:
        VMT_t7_data_rows.append(row)
        
VMT_t7 = pd.DataFrame(VMT_t7_data_rows, columns = header7)
VMT_t7

In [ ]:
# Combine the data frames

VMT_final = VMT_t6.append(VMT_t7, ignore_index = True)
VMT_final

In [ ]:
# Convert long date to short date

VMT_final['Date'] = pd.to_datetime(VMT_final['Date'])

In [ ]:
# Add year and month columns

VMT_final['Year'] = pd.DatetimeIndex(VMT_final['Date']).year
VMT_final['Month'] = pd.DatetimeIndex(VMT_final['Date']).month

In [ ]:
# Rename "Value" to VMT

VMT_final.rename(columns = {"Value": "VMT_in_Millions"}, inplace=True)

In [ ]:
# Determine current data types
VMT_final.dtypes

In [ ]:
# Format VMT in Millions to include a comma

VMT_final["VMT_in_Millions"] = pd.to_numeric(VMT_final["VMT_in_Millions"])

In [ ]:
# Check data type

VMT_final.dtypes

In [ ]:
# Add comma to VMT in Millions and remove the decimal point

pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
VMT_final

In [ ]:
VMT_final.set_index("Date",drop=False,inplace=True)

In [ ]:
VMT_final

In [ ]:
# Send data to CSV

VMT_final.to_csv("output/VMT_final.csv")

In [ ]:
VMT_final.dtypes

In [ ]:
# Not sure why I would need this when I added Year and Month to the main data frame. Commenting it out for now, may come back to it.

# Create yearly data frames to do some year over year comparisons

# VMT_2017 = VMT_final.loc[39:49] # does not contain January 
# VMT_2018 = VMT_final.loc[38:27] # January - December
# VMT_2019 = VMT_final.loc[15:26] # January - December
# VMT_2020 = VMT_final.loc[3:14] # January - December
# VMT_2021_YTD = VMT_final.loc[0:2] # January - March

# EDA

#### Right now I have my data frame(s) where I want it. I'll edit as needed throughout the workbook but I think I am ready for some data analysis. Let's start with some statistics.

#### I am not expecting sweetviz to show anything helpful, but I liked it when we used it in HW1 so let's see what the output is.

In [ ]:
report = sweetviz.analyze(VMT_final)
report.show_html("output/sweetviz_report.html")

#### As expected, not great but it does tell me I have missing values (which I already knew since I don't have data for April - December in 2021 but all other years have this data). Sweetviz did show some association between Month and VMT which we already gathered from the charts above. Since sweetviz was not too insightful let's do our own data analysis.

In [ ]:
# Look at basic statistics by year

summary_stats_year = VMT_final.groupby(['Year'])['VMT_in_Millions']
summary_stats_year.describe()

In [ ]:
summary_stats_year.describe().to_csv("output/year_summary.csv") # I thought this might be cool to use as a widget but I was wrong

#### We have three full years of data between 2018 - 2020 so just looking at those years, you can see how Covid impacted VMT.

#### Another way to look at the data would be to reivew stats by month rather than year. 

In [ ]:
# Look at basic statistics by month

summary_stats_month = VMT_final.groupby(['Month'])['VMT_in_Millions']
summary_stats_month.describe()

In [ ]:
summary_stats_month.describe().to_csv("output/month_summary.csv") # I thought this might be cool to use as a widget but I was wrong

#### The average VMT by month looks normal but the standard deviation in March, April, and May are drastically different than other months. These months were impacted by Covid so it is probably a driving factor within these months. On average, May - August appear to have the most miles traveled. This would make sense due to spring break and summer vacations.

#### .describe() does not show the median so I actually went back to my sweetviz output and the overal median VMT is 260K which is in line with the monthly statistics output. The boxplot below (as well as .median()) also shows a similar median. The boxplot also shows the outlier in 2020 when VMT fell to approx. 165K in April 2020.

In [ ]:
# Create boxplot to show median

sns.boxplot(data=VMT_final["VMT_in_Millions"])

In [ ]:
VMT_final['VMT_in_Millions'].median()

#### Some basic insights / things to note here:

-  2017 is only February - December
-  2018 - 2020 is January - December
-  2021 is only January - March
-  Avg VMT sat around 230M / month until it dropped in 2020

In [ ]:
# Creating a histogram to see how the data is distributed as a whole

sns.displot(kde=True,bins=5,data=VMT_final['VMT_in_Millions'])

# VMT tends to be +260K

#### Let's see what the data actually looks like by using some charts.

In [ ]:
fig, ax1 = plt.subplots(figsize = (20,5))

# Plot line graph

chart1 = sns.lineplot(x = "Date", y = "VMT_in_Millions", data = VMT_final)

# Format x-axis


ax1.set_xlabel("Date" , size = 14)
ax1.set(xticks=VMT_final.Date.values)
chart1.set_xticklabels(labels = VMT_final['Date'].dt.strftime('%Y-%m'), rotation=45)

# Format y-axis

ax1.set_ylabel("VMT (in Millions)" , size = 14)
ax1.yaxis.set_major_formatter('{x:1,.0f}')

# Create chart title

ax1.set_title( "Vehicle Miles Traveled (VMT)" , size = 16)

#### VMT appears to be seasonal; around November/December, VMT starts to decline but it could be due to the holiday season. VMT picks back up in January in to April which could be due to vacations (especially March/April as those are spring break months). These trends were consistent in 2017 to 2019 but once March 2020 hit and the lockdowns started to happen, VMT took a nose dive. VMT did not recover until late summer but then followed the same decline in November/December and started to rebound at the new year.

#### I'll plot the seasonality for kicks and giggles.

In [ ]:
# Pivot the data frame to a wide-form representation of the data

VMT_wide = VMT_final.pivot("Month", "Year", "VMT_in_Millions") # did not do .fillna() because it messed with my graph -
                                                                # it did not include any columns with missing data
VMT_wide

In [ ]:
fig, ax2 = plt.subplots(figsize = (10,5))

# Plot line graph

sns.lineplot(data=VMT_wide)

# Format y-axis

ax2.set_ylabel("VMT (in Millions)", size = 12)
ax2.yaxis.set_major_formatter('{x:1,.0f}')

# Create chart title

ax2.set_title( "Vehicle Miles Traveled (VMT) by Year" , size = 16)

#### Really interesting how VMT between 2017 - 2021 basically followed the same tread year over year; even with Covid, the 2020 trendline was extremely similar to other years. VMT grew every year but no major changes to the actual trendline.

#### I did replace the missing values with the median but all it did was create a flat line for April - December 2021 and I didn't like the look so I took it out. If I was a real whiz with Python, I would take the average VMT by month (probably excluding 2020 because it is an an anomoly) and put that in for 2021 missing values.

# Enter widgets...I tried, I promise.

#### I tried over and over again to get this widget to work but I can't. I didn't use my VMT_final data frame because I did not want someone to be able to select VMT as a selector. I tried to create a chart that allowed you to look at one year and X months at a time. I even tried to use widget.SelectMultiple so you could choose multiple years but I didn't work and maybe it does not like the VMT_wide data frame because I transformed it in to a pivot.

In [ ]:
# Create year drop down widget

year_data_cols = VMT_wide.columns.to_list()

year_range = widgets.Dropdown(
    options=year_data_cols,
    description='Year'
)

In [ ]:
# Create month range widget

style = {'description_width': 'initial'}

month_range = widgets.FloatRangeSlider(
    value=[1, 12],
    min=1,
    max=12,
    step=1,
    description=r'Month',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
    style=style
)

In [ ]:
# Create interactive plot

def dates_plot(year, month):
    
    # sns.lineplot(data=VMT_wide) # it doesnt work with this line either
    plt.rcParams["figure.figsize"] = (10,3)
    plt.figure(1)
    plt.plot(VMT_wide) 
    plt.xlabel('Month')
    plt.ylabel('VMT')
    plt.title('Vehcile Miles Traveled (VMT) by Year')
    plt.show()

interactive_plot = interactive(dates_plot, year=year_range, month=month_range);
interactive_plot

#### This widget is not as "complex" as the one above but it works for year using the VMT_wide data frame. However if I were an end user, I would rather look at the initial line graphs I created rather than use this widget...had I been able to figure out how to do the above widget, it would probably be more helpful/interesting/insightful.

In [ ]:
# Create year drop down widget

year_data_cols_2 = VMT_wide.columns.to_list()

year_range_2 = widgets.Dropdown(
    options=year_data_cols_2,
    description='Year'
)

# Create interactive plot 

def create_line_2(year_2):
    plt.rcParams["figure.figsize"] = (10,3)
    plt.plot(VMT_wide[year_2])
    plt.xlabel('Month')
    plt.ylabel('VMT')
    plt.title('Vehicle Miles Traveled by Year')
    plt.show()
    
widgets.interact(create_line_2, year_2=year_range_2);

#### I was pretty bummed about the widget not working so I went to Google to look up some other data analysis and I found someone who did a rolling average interaction and I decided to try it. They used "@ interact" rather than the interact/interactive method.

In [ ]:
@ interact

# n indicates month, I have 50 months

def plot(n=(1, 49)):
        fig, ax = plt.subplots(1, 1, figsize=(12,4))
        VMT_final['VMT_in_Millions'].rolling(window=n).mean().plot(ax=ax)
        ax.set_title( "Vehicle Miles Traveled (VMT) - Rolling Average" , size = 14)
        ax.set_ylabel("VMT (in Millions)", size = 12)
        ax.set_ylim(150000, 300000)
        ax.autoscale(enable=True, axis='x', tight=True)
        ax.yaxis.set_major_formatter('{x:1,.0f}')
        fig.autofmt_xdate()
        plt.show()

#### So if you start at n=1 you'll see the standard chart with the actual data. However once you move to n=3, the line/data for 2021 drops off because a rolling average cannot be calculated (that was my test to make sure it worked).

#### 2017 will appear on the X axis once n hits 40; this is because I have 'tight=True.' When I change it to 'tight=False,' the chart starts at January 2017 but my data does not start until February 2017 so 'tight=True' kept the x-axis from going back to January if you set n equal to 49.